In [1]:
from paddleocr import PaddleOCR,draw_ocr
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [38]:
test_df = pd.read_csv('DataTestLabelled.csv')

In [39]:
test_df.head()

,Unnamed: 0,Name of File,Image Labels
0,0,DataTest1.png,AD7034OE
1,1,DataTest2.png,A9388EX
2,2,DataTest3.png,B16TB
3,3,DataTest4.png,B1661TKZ
4,4,DataTest5.png,AD3772ABE


In [27]:
ocr = PaddleOCR(use_angle_cls=True, lang='en') # need to run only once to download and load model into memory

[2023/07/06 04:41:02] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/rsoedarnadi/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/rsoedarnadi/.paddleocr/whl/rec/en/en_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_

In [52]:
generated_plate_numbers = []
for file_name in test_df['Name of File']:
    img_path = os.path.join('Test-Data',file_name)
    result = ocr.ocr(img_path, cls=True)
    concat_output = "".join(row[1][0] for row in result[0])
    generated_plate_numbers.append(concat_output)
    

[2023/07/06 04:53:23] ppocr DEBUG: dt_boxes num : 1, elapse : 0.10521602630615234
[2023/07/06 04:53:23] ppocr DEBUG: cls num  : 1, elapse : 0.019558191299438477
[2023/07/06 04:53:24] ppocr DEBUG: rec_res num  : 1, elapse : 0.2281332015991211
[2023/07/06 04:53:24] ppocr DEBUG: dt_boxes num : 2, elapse : 0.03541898727416992
[2023/07/06 04:53:24] ppocr DEBUG: cls num  : 2, elapse : 0.022155046463012695
[2023/07/06 04:53:24] ppocr DEBUG: rec_res num  : 2, elapse : 0.2098240852355957
[2023/07/06 04:53:24] ppocr DEBUG: dt_boxes num : 1, elapse : 0.030736207962036133
[2023/07/06 04:53:24] ppocr DEBUG: cls num  : 1, elapse : 0.014200210571289062
[2023/07/06 04:53:24] ppocr DEBUG: rec_res num  : 1, elapse : 0.07959604263305664
[2023/07/06 04:53:24] ppocr DEBUG: dt_boxes num : 1, elapse : 0.03889298439025879
[2023/07/06 04:53:24] ppocr DEBUG: cls num  : 1, elapse : 0.012404918670654297
[2023/07/06 04:53:24] ppocr DEBUG: rec_res num  : 1, elapse : 0.07839584350585938
[2023/07/06 04:53:24] ppocr D

In [53]:
for i, platenum in enumerate(generated_plate_numbers):
   generated_plate_numbers[i] = re.sub('[^A-Za-z0-9]+', '', platenum)

In [54]:
list_license_plates = test_df['Image Labels']
list_file_names = test_df['Name of File']

In [55]:
def calculate_predicted_accuracy(names, actual_list, predicted_list):
    list_accuracy = []
    for name, actual_plate, predict_plate in zip(names, actual_list, predicted_list):
        accuracy = 0
        num_matches = 0
        if actual_plate == predict_plate:
            accuracy = 1.0
        else:
            for a, p in zip(actual_plate, predict_plate):
                if a == p:
                    num_matches += 1
            accuracy = round((num_matches / len(actual_plate)), 2)
        list_accuracy.append(accuracy)
    list_accuracy = np.asarray(list_accuracy)
    avg_value = np.average(list_accuracy)
    return avg_value

In [56]:
calculate_predicted_accuracy(list_file_names, list_license_plates, generated_plate_numbers)

0.48029999999999995